<a href="https://colab.research.google.com/github/chloebh9/Recommendation-system/blob/main/L08_1_Factorization_Machines_Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## FastFM

In [ ]:
!pip install fastfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fastfm: filename=fastFM-0.2.10-cp310-cp310-linux_x86_64.whl size=591640 sha256=475a8464a17026c3bf55febda550cd89c8428a3810d2421a97b56bc01c2ea6d6
  Stored in directory: /root/.cache/pip/wheels/93/92/52/2da7997fcb7a7ce9042ff3b33836ef0c2fd47aa95382d7a113
Successfully built fastfm


In [ ]:
!wget https://datarepo.eng.ucsd.edu/mcauley_group/gdrive/goodreads/byGenre/goodreads_reviews_comics_graphic.json.gz

--2024-06-01 15:06:50--  https://datarepo.eng.ucsd.edu/mcauley_group/gdrive/goodreads/byGenre/goodreads_reviews_comics_graphic.json.gz
Resolving datarepo.eng.ucsd.edu (datarepo.eng.ucsd.edu)... 132.239.8.30
Connecting to datarepo.eng.ucsd.edu (datarepo.eng.ucsd.edu)|132.239.8.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 146582543 (140M) [application/x-gzip]
Saving to: ‘goodreads_reviews_comics_graphic.json.gz’

goodreads_reviews_c 100%[===================>] 139.79M   932KB/s    in 2m 37s  

2024-06-01 15:09:35 (909 KB/s) - ‘goodreads_reviews_comics_graphic.json.gz’ saved [146582543/146582543]



In [ ]:
import gzip
user_ids = {}
book_ids = {}
data = []

with gzip.open("goodreads_reviews_comics_graphic.json.gz", "rt") as f:
    for line in f:
        record = eval(line)

        uid = record['user_id']
        bid = record['book_id']
        rating = record['rating']

        if uid not in user_ids:
            user_ids[uid] = len(user_ids)
        if bid not in book_ids:
            book_ids[bid] = len(book_ids)

        data.append( (user_ids[uid], book_ids[bid], rating) )
n_users = len(user_ids)
n_books = len(book_ids)

In [ ]:
from scipy.sparse import csr_matrix
import numpy as np
import random

random.shuffle(data)

rows = []
cols = []
values = []

for rowid, (uid, bid, rating) in enumerate(data):
    rows.extend([rowid, rowid])
    cols.extend([uid, n_users+bid])
    values.extend([1,1])

X = csr_matrix( (values, (rows, cols)) )
y = np.array([rating for _, _, rating in data])

X_train, y_train = X[:400000], y[:400000]
X_test, y_test = X[400000:], y[400000:]

### FastFM 학습

In [ ]:
from fastFM import als

fm = als.FMRegression(n_iter=1000, init_stdev=0.1, rank=5, l2_reg_w=0.1, l2_reg_V=0.5)
fm.fit(X_train, y_train)

FMRegression(l2_reg_V=0.5, n_iter=1000, rank=5)

In [ ]:
y_pred = fm.predict(X_test)
print(y_pred[:10])
print(y_test[:10])
print("MSE: ", ((y_test-y_pred)**2).mean() ** 0.5)

[3.61845292 3.06981892 4.3655838  2.24447102 4.72477278 3.93296067
 4.9479025  4.44765616 3.54348201 2.71869826]
[3 3 5 2 5 5 5 4 5 3]
MSE:  1.2650846619646723
